In [1]:
import os
import numpy as np
import pandas as pd
from datetime import date

from sklearn.model_selection import KFold, train_test_split, StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import log_loss, roc_auc_score, auc, roc_curve
from sklearn.preprocessing import MinMaxScaler

DATA_ROOT = "../data/median_test_data/"

In [2]:
dfoff = pd.read_csv(os.path.join(DATA_ROOT,'train_offline.csv'))
dftest = pd.read_csv(os.path.join(DATA_ROOT,'test_offline.csv'))
dftest = dftest[~dftest.Coupon_id.isna()]
dftest.reset_index(drop=True, inplace=True)
print(dfoff.shape)
print(dftest.shape)
dfoff.head(20)

(1160742, 7)
(306313, 6)


,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date
0,1439408,2632,NaN,NaN,0.0,NaN,20160217.0
1,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN
2,1439408,2632,1078.0,20:1,0.0,20160319.0,NaN
3,1832624,3381,7610.0,200:20,0.0,20160429.0,NaN
4,2029232,3381,11951.0,200:20,1.0,20160129.0,NaN
5,2223968,3381,9776.0,10:5,2.0,20160129.0,NaN
6,73611,2099,12034.0,100:10,NaN,20160207.0,NaN
7,163606,1569,5054.0,200:30,10.0,20160421.0,NaN
8,3273056,4833,7802.0,200:20,10.0,20160130.0,NaN
9,94107,3381,7610.0,200:20,2.0,20160412.0,NaN


In [3]:
dftest.head(20)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received
0,1439408,4663,11002.0,150:20,1.0,20160528.0
1,1439408,2632,8591.0,20:1,0.0,20160613.0
2,1439408,2632,8591.0,20:1,0.0,20160516.0
3,2029232,450,1532.0,30:5,0.0,20160530.0
4,2029232,6459,12737.0,20:1,0.0,20160519.0
5,2747744,6901,1097.0,50:10,NaN,20160606.0
6,196342,1579,10698.0,20:1,1.0,20160606.0
7,253750,6901,2366.0,30:5,0.0,20160518.0
8,343660,4663,11002.0,150:20,NaN,20160528.0
9,1113008,3621,2705.0,20:5,0.0,20160524.0


In [4]:
## Creat target label 
"""
According to the definition, 
1) buy with coupon within (include) 15 days ==> 1
2) buy with coupon but out of 15 days ==> 0
3) buy without coupon ==> -1 (we don't care)
"""
def label(row):
    if np.isnan(row['Date_received']):
        return -1
    if not np.isnan(row['Date']):
        td = pd.to_datetime(row['Date'], format='%Y%m%d') -  pd.to_datetime(row['Date_received'], format='%Y%m%d')
        if td <= pd.Timedelta(15, 'D'):
            return 1
    return 0

dfoff["label"] = dfoff.apply(label, axis=1)
dfoff["label"].value_counts()


 0    710665
-1    413773
 1     36304
Name: label, dtype: int64

In [5]:
# 後面多加一行 label欄位
# Date - Date_received 要是小於15天則標註 1, 大於15天標註 0, 沒有 Date_received (表示沒發卷給消費者)標註 -1
dfoff.head(20)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,label
0,1439408,2632,NaN,NaN,0.0,NaN,20160217.0,-1
1,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN,0
2,1439408,2632,1078.0,20:1,0.0,20160319.0,NaN,0
3,1832624,3381,7610.0,200:20,0.0,20160429.0,NaN,0
4,2029232,3381,11951.0,200:20,1.0,20160129.0,NaN,0
5,2223968,3381,9776.0,10:5,2.0,20160129.0,NaN,0
6,73611,2099,12034.0,100:10,NaN,20160207.0,NaN,0
7,163606,1569,5054.0,200:30,10.0,20160421.0,NaN,0
8,3273056,4833,7802.0,200:20,10.0,20160130.0,NaN,0
9,94107,3381,7610.0,200:20,2.0,20160412.0,NaN,0


In [6]:
# Generate features - weekday acquired coupon
def getWeekday(row):
    if (np.isnan(row)) or (row==-1):
        return row
    else:
        return pd.to_datetime(row, format = "%Y%m%d").dayofweek+1 # add one to make it from 0~6 -> 1~7

#增加 星期幾 使用特價劵的欄位
dfoff['weekday'] = dfoff['Date_received'].apply(getWeekday)
dftest['weekday'] = dftest['Date_received'].apply(getWeekday)

# weekday_type (weekend = 1)
dfoff['weekday_type'] = dfoff['weekday'].astype('str').apply(lambda x : 1 if x in [6,7] else 0 ) # apply to trainset
dftest['weekday_type'] = dftest['weekday'].astype('str').apply(lambda x : 1 if x in [6,7] else 0 ) # apply to testset

In [7]:
# 後面多加一行 label欄位
# Date - Date_received 要是小於15天則標註 1, 大於15天標註 0, 沒有 Date_received (表示沒發卷給消費者)標註 -1
dfoff.head(10)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,label,weekday,weekday_type
0,1439408,2632,NaN,NaN,0.0,NaN,20160217.0,-1,NaN,0
1,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN,0,3.0,0
2,1439408,2632,1078.0,20:1,0.0,20160319.0,NaN,0,6.0,0
3,1832624,3381,7610.0,200:20,0.0,20160429.0,NaN,0,5.0,0
4,2029232,3381,11951.0,200:20,1.0,20160129.0,NaN,0,5.0,0
5,2223968,3381,9776.0,10:5,2.0,20160129.0,NaN,0,5.0,0
6,73611,2099,12034.0,100:10,NaN,20160207.0,NaN,0,7.0,0
7,163606,1569,5054.0,200:30,10.0,20160421.0,NaN,0,4.0,0
8,3273056,4833,7802.0,200:20,10.0,20160130.0,NaN,0,6.0,0
9,94107,3381,7610.0,200:20,2.0,20160412.0,NaN,0,2.0,0


In [8]:
# 建造欄位標籤
weekdaycols = ['weekday_' + str(i) for i in range(1,8)]
print(weekdaycols)

['weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6', 'weekday_7']


In [9]:
# 用 One-Hot 編碼編制 星期幾
tmpdf = pd.get_dummies(dfoff['weekday'].replace(-1, np.nan))
# print(tmpdf.head())

# 將方才建的欄位標籤 新增至 tmpdf 表格內
tmpdf.columns = weekdaycols
# print(tmpdf.head())

# 將 tmpdf 加入到 dfoff
dfoff[weekdaycols] = tmpdf
# print(dfoff.head(10))

# dftest 做與 dfoff 一樣的動作
tmpdf = pd.get_dummies(dftest['weekday'].replace(-1, np.nan))
tmpdf.columns = weekdaycols
dftest[weekdaycols] = tmpdf

In [10]:
# 新增 tmpdf 後的結果
dfoff.head(10)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,label,weekday,weekday_type,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weekday_7
0,1439408,2632,NaN,NaN,0.0,NaN,20160217.0,-1,NaN,0,0,0,0,0,0,0,0
1,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN,0,3.0,0,0,0,1,0,0,0,0
2,1439408,2632,1078.0,20:1,0.0,20160319.0,NaN,0,6.0,0,0,0,0,0,0,1,0
3,1832624,3381,7610.0,200:20,0.0,20160429.0,NaN,0,5.0,0,0,0,0,0,1,0,0
4,2029232,3381,11951.0,200:20,1.0,20160129.0,NaN,0,5.0,0,0,0,0,0,1,0,0
5,2223968,3381,9776.0,10:5,2.0,20160129.0,NaN,0,5.0,0,0,0,0,0,1,0,0
6,73611,2099,12034.0,100:10,NaN,20160207.0,NaN,0,7.0,0,0,0,0,0,0,0,1
7,163606,1569,5054.0,200:30,10.0,20160421.0,NaN,0,4.0,0,0,0,0,1,0,0,0
8,3273056,4833,7802.0,200:20,10.0,20160130.0,NaN,0,6.0,0,0,0,0,0,0,1,0
9,94107,3381,7610.0,200:20,2.0,20160412.0,NaN,0,2.0,0,0,1,0,0,0,0,0


In [11]:
# Generate features - coupon discount and distance
# Discount_rate 欄位 Nan 給 0, 其餘給 1
def getDiscountType(row):
    if row == 'null':
        return 'null'
    elif ':' in row:
        return 1
    else:
        return 0

# 將 Discount_rate 欄位轉化成小數點 
def convertRate(row):
    """Convert discount to rate"""
    if row == 'null':
        return 1.0
    elif ':' in row:
        rows = row.split(':')
        return 1.0 - float(rows[1])/float(rows[0])
    else:
        return float(row)

# 將 Discount_rate 欄位的分母保留
def getDiscountMan(row):
    if ':' in row:
        rows = row.split(':')
        return int(rows[0])
    else:
        return 0

# Discount_rate 欄位的分子保留
def getDiscountJian(row):
    if ':' in row:
        rows = row.split(':')
        return int(rows[1])
    else:
        return 0

# 新增 discount_rate, discount_man, discount_jian, discount_type 欄位的 function
def processData(df):
    # convert discunt_rate
    df['discount_rate'] = df['Discount_rate'].astype('str').apply(convertRate)
    df['discount_man'] = df['Discount_rate'].astype('str').apply(getDiscountMan)
    df['discount_jian'] = df['Discount_rate'].astype('str').apply(getDiscountJian)
    df['discount_type'] = df['Discount_rate'].astype('str').apply(getDiscountType)
    # convert distance - 假如 distance欄位有 Nan則補 99
    df.loc[df.Distance.isna(), "Distance"] = 99
    return df

dfoff = processData(dfoff)
dftest = processData(dftest)

In [12]:
# 新增 discount_rate, discount_man, discount_jian, discount_type 四個欄位的結果
dfoff.head(10)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,label,weekday,weekday_type,...,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weekday_7,discount_rate,discount_man,discount_jian,discount_type
0,1439408,2632,NaN,NaN,0.0,NaN,20160217.0,-1,NaN,0,...,0,0,0,0,0,0,NaN,0,0,0
1,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN,0,3.0,0,...,0,1,0,0,0,0,0.95,20,1,1
2,1439408,2632,1078.0,20:1,0.0,20160319.0,NaN,0,6.0,0,...,0,0,0,0,1,0,0.95,20,1,1
3,1832624,3381,7610.0,200:20,0.0,20160429.0,NaN,0,5.0,0,...,0,0,0,1,0,0,0.90,200,20,1
4,2029232,3381,11951.0,200:20,1.0,20160129.0,NaN,0,5.0,0,...,0,0,0,1,0,0,0.90,200,20,1
5,2223968,3381,9776.0,10:5,2.0,20160129.0,NaN,0,5.0,0,...,0,0,0,1,0,0,0.50,10,5,1
6,73611,2099,12034.0,100:10,99.0,20160207.0,NaN,0,7.0,0,...,0,0,0,0,0,1,0.90,100,10,1
7,163606,1569,5054.0,200:30,10.0,20160421.0,NaN,0,4.0,0,...,0,0,1,0,0,0,0.85,200,30,1
8,3273056,4833,7802.0,200:20,10.0,20160130.0,NaN,0,6.0,0,...,0,0,0,0,1,0,0.90,200,20,1
9,94107,3381,7610.0,200:20,2.0,20160412.0,NaN,0,2.0,0,...,1,0,0,0,0,0,0.90,200,20,1


In [13]:
# 將 '有發到優惠劵' 的資料另外存起來
# train 時指對有拿到劵的人是否在15天內用到做預測
df_train = dfoff[dfoff['label'] != -1].copy()
train_Y = df_train['label']

df_test = dftest.copy()

# 因為 df_test 沒有 label 欄位,所以在合併前先把 df_train內的 label欄位拿掉
df_train.drop(labels = ["label"], axis = 1, inplace = True)
df = pd.concat([df_train,df_test])

# 捨棄 Name 欄位
df.drop(labels = ["Discount_rate"], axis = 1, inplace = True)
df.drop(labels = ["Date_received"], axis = 1, inplace = True)
df.drop(labels = ["Date"], axis = 1, inplace = True)

#print(df)

# 將資料最大最小化
df = MinMaxScaler().fit_transform(df)
#print (df)

# 將前述轉換完畢資料 df , 重新切成 train_X, test_X
train_num = train_Y.shape[0]
#print(train_num)

# L[1:],表示第二個元素開始截取列, L[:5], 表示從頭擷取到第五個元素
train_X = df[:train_num]
test_X = df[train_num:]
#print (train_X)
#print (test_X)

# 使用三種模型 : 邏輯斯迴歸 / 梯度提升機 / 隨機森林, 參數使用 Random Search 尋找
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
lr = LogisticRegression(tol=0.001, penalty='l2', fit_intercept=True, C=1.0)
gdbt = GradientBoostingClassifier(tol=100, subsample=0.75, n_estimators=250, max_features=1.0,
                                  max_depth=6, learning_rate=0.03)
rf = RandomForestClassifier(n_estimators=100, min_samples_split=2, min_samples_leaf=1, 
                            max_features='sqrt', max_depth=6, bootstrap=True)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # Remove the CWD from sys.path while we load stuff.
D:\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [14]:
print(train_X)

[[0.6116491  0.         0.29704089 ... 0.         0.         0.        ]
 [0.07668755 0.         0.29704089 ... 1.         0.         0.        ]
 [0.54179721 0.         0.38163542 ... 0.         0.         0.        ]
 ...
 [0.4047992  0.02020202 0.33114976 ... 0.         0.         0.        ]
 [0.3749644  0.01010101 0.39868986 ... 0.         0.         0.        ]
 [0.4047992  0.02020202 0.33114976 ... 0.         0.         0.        ]]


In [15]:
print(train_Y)

1          0
2          0
3          0
4          0
5          0
6          0
7          0
8          0
9          0
11         0
12         0
14         0
16         0
17         0
18         1
19         0
21         0
22         0
23         0
24         0
26         0
27         0
28         0
29         0
30         0
31         0
32         0
33         0
34         0
35         0
          ..
1160698    0
1160699    0
1160701    1
1160703    0
1160704    0
1160705    0
1160706    0
1160707    0
1160709    1
1160710    0
1160711    0
1160713    0
1160715    0
1160717    0
1160718    0
1160719    0
1160720    0
1160721    0
1160722    0
1160723    0
1160724    0
1160725    0
1160726    0
1160727    0
1160728    0
1160732    0
1160734    0
1160737    1
1160738    0
1160739    1
Name: label, Length: 746969, dtype: int64


In [16]:
print(test_X)

[[0.78332384 0.01010101 0.52642873 ... 1.         0.         0.        ]
 [0.6116491  0.         0.29704089 ... 0.         0.         0.        ]
 [0.6116491  0.         0.29704089 ... 0.         0.         0.        ]
 ...
 [0.26616349 0.06060606 0.34097583 ... 0.         1.         0.        ]
 [0.11620621 0.06060606 0.80313982 ... 0.         0.         0.        ]
 [0.19253774 0.         0.40874181 ... 0.         0.         0.        ]]


In [17]:
# 線性迴歸預測檔 (結果有部分隨機, 請以 Kaggle 計算的得分為準, 以下模型同理)
lr.fit(train_X, train_Y)
lr_pred = lr.predict_proba(test_X)[:,1]

ids = df_test['User_id']
lr_sub = pd.DataFrame({'User_id': ids, 'label': lr_pred})
#sub['label'] = sub['label'].map(lambda x:1 if x>0.5 else 0) 
lr_sub.to_csv('MedianTest_lr.csv', index=False) 

D:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [20]:
# 梯度提升機預測檔 
gdbt.fit(train_X, train_Y)
gdbt_pred = gdbt.predict_proba(test_X)[:,1]

ids = df_test['User_id']
gdbt_sub = pd.DataFrame({'User_id': ids, 'label': gdbt_pred})
#gdbt_sub['label'] = gdbt_sub['label'].map(lambda x:1 if x>0.5 else 0) 
gdbt_sub.to_csv('MedianTest_gdbt.csv', index=False) 

In [19]:
# 隨機森林預測檔
rf.fit(train_X, train_Y)
rf_pred = rf.predict_proba(test_X)[:,1]

ids = df_test['User_id']
rf_sub = pd.DataFrame({'User_id': ids, 'label': rf_pred})
#rf_sub['label'] = rf_sub['label'].map(lambda x:1 if x>0.1 else 0) 
rf_sub.to_csv('MedianTest_rf.csv', index=False) 

In [21]:
# 混合泛化預測檔 
ids = df_test['User_id']
blending_pred = lr_pred*0.30 + gdbt_pred*0.67 + rf_pred*0.03
blending_sub = pd.DataFrame({'User_id': ids, 'label': blending_pred})
#blending_sub['label'] = blending_sub['label'].map(lambda x:1 if x>0.5 else 0) 
blending_sub.to_csv('MedianTest_blending.csv', index=False)

print(valid.head())

# 新增一個欄位結構 original_feature, 並加上之前的欄位結構 weekdaycols
original_feature = ['discount_rate',
                    'discount_type',
                    'discount_man', 
                    'discount_jian',
                    'Distance', 
                    'weekday', 
                    'weekday_type'] + weekdaycols
print(len(original_feature),original_feature)

predictors = original_feature
print(predictors)

# Check model 由以下內建 model組成
# from sklearn.model_selection import KFold, train_test_split, StratifiedKFold, cross_val_score, GridSearchCV
# from sklearn.pipeline import Pipeline
# from sklearn.linear_model import SGDClassifier, LogisticRegression
# from sklearn.preprocessing import StandardScaler

def check_model(data, predictors):
    
    classifier = lambda: SGDClassifier(
        loss='log', 
        penalty='elasticnet', 
        fit_intercept=True, 
        max_iter=100, 
        shuffle=True, 
        n_jobs=1,
        class_weight=None)

    model = Pipeline(steps=[
        ('ss', StandardScaler()),
        ('en', classifier())
    ])

    parameters = {
        'en__alpha': [ 0.001, 0.01, 0.1],
        'en__l1_ratio': [ 0.001, 0.01, 0.1]
    }

    folder = StratifiedKFold(n_splits=3, shuffle=True)
    
    grid_search = GridSearchCV(
        model, 
        parameters, 
        cv=folder, 
        n_jobs=-1, 
        verbose=1)
    grid_search = grid_search.fit(data[predictors], 
                                  data['label'])
    
    return grid_search

model = check_model(train, predictors)

# model 為 Pipeline 內建的 class
y_valid_pred = model.predict_proba(valid[predictors])
valid1 = valid.copy()

# 另外存一份 valid1 並在後面加上 'pred_prob' 欄位
valid1['pred_prob'] = y_valid_pred[:, 1]
valid1.head()

# 計算分數 AUC, ACC
from sklearn.metrics import roc_auc_score, accuracy_score
auc_score = roc_auc_score(y_true=valid.label, y_score=y_valid_pred[:,1])
acc = accuracy_score(y_true=valid.label, y_pred=y_valid_pred.argmax(axis=1))
print("Validation AUC: {:.3f}, Accuracy: {:.3f}".format(auc_score, acc))

dftest.head(30)

targetset = dftest.copy()
print(targetset.shape)
targetset = targetset[~targetset.Coupon_id.isna()]
targetset.reset_index(drop=True, inplace=True)
testset = targetset[predictors].copy()

# predictors 為 original_feature 欄位 + weekdaycols 欄位 
print(testset.head())

y_test_pred = model.predict_proba(testset[predictors])
test1 = testset.copy()
# 在 test1 後面新增 'pred_prob' 欄位
test1['pred_prob'] = y_test_pred[:, 1]
print(test1.shape)

In [22]:
targetset = dftest.copy()
print(targetset.shape)
targetset = targetset[~targetset.Coupon_id.isna()]
targetset.reset_index(drop=True, inplace=True)

(306313, 19)


In [31]:
# 新增 output 資料欄位
#output = pd.concat((targetset[["User_id", "Coupon_id", "Date_received"]], blending_sub['label'] ), axis=1)
output = pd.concat((targetset[["User_id", "Coupon_id", "Date_received"]], lr_sub['label'] ), axis=1)
#output = pd.concat((targetset[["User_id", "Coupon_id", "Date_received"]], gdbt_sub['label'] ), axis=1)
#output = pd.concat((targetset[["User_id", "Coupon_id", "Date_received"]], rf_sub['label'] ), axis=1)


print(output.shape)
print(output.head())

# apply(lambda x:str(int(x))) 將原本 int 改成 string
output.loc[:, "User_id"] = output["User_id"].apply(lambda x:str(int(x)))
output.loc[:, "Coupon_id"] = output["Coupon_id"].apply(lambda x:str(int(x)))
output.loc[:, "Date_received"] = output["Date_received"].apply(lambda x:str(int(x)))

#新增一個 uid 欄位, 為 "User_id", "Coupon_id", "Date_received" 這三者的結合字串
output["uid"] = output[["User_id", "Coupon_id", "Date_received"]].apply(lambda x: '_'.join(x.values), axis=1)
output.reset_index(drop=True, inplace=True)

(306313, 4)
   User_id  Coupon_id  Date_received     label
0  1439408    11002.0     20160528.0  0.013675
1  1439408     8591.0     20160613.0  0.069874
2  1439408     8591.0     20160516.0  0.069874
3  2029232     1532.0     20160530.0  0.050236
4  2029232    12737.0     20160519.0  0.129941


In [27]:
print(output.head())

   User_id Coupon_id Date_received     label                     uid
0  1439408     11002      20160528  0.014688  1439408_11002_20160528
1  1439408      8591      20160613  0.133827   1439408_8591_20160613
2  1439408      8591      20160516  0.133827   1439408_8591_20160516
3  2029232      1532      20160530  0.109845   2029232_1532_20160530
4  2029232     12737      20160519  0.158375  2029232_12737_20160519


In [32]:
### NOTE: YOUR SUBMITION FILE SHOULD HAVE COLUMN NAME: uid, label
# groupby 用法
# 根據 uid 欄位內的 id, 重複的 id 其對應的數值欄位 (目前只剩下 'pred_prob') 做平均
out = output.groupby("uid", as_index=False).mean()
print(out.head())

# out 的兩個欄位 "uid", "pred_prob"
out = out[["uid", "label"]]
# 改 out 的兩個欄位名稱 "uid", "label"
# out.columns = ["uid", "label"]
#out.to_csv("blending_sub.csv", header=["uid", "label"], index=False) # submission format
out.to_csv("lr_sub.csv", header=["uid", "label"], index=False) # submission format
#out.to_csv("gdbt_sub.csv", header=["uid", "label"], index=False) # submission format
#out.to_csv("rf_sub.csv", header=["uid", "label"], index=False) # submission format

                     uid     label
0  1000020_2705_20160519  0.107717
1  1000020_8192_20160513  0.097055
2  1000065_1455_20160527  0.058985
3  1000085_8067_20160513  0.069809
4  1000086_2418_20160613  0.051767


In [42]:
out.head()

,uid,label
0,1000020_2705_20160519,0.113804
1,1000020_8192_20160513,0.088059
2,1000065_1455_20160527,0.067609
3,1000085_8067_20160513,0.070512
4,1000086_2418_20160613,0.060913
